Needed imports and utility

In [ ]:
import os
import ansys.speos.core as core
import time

tests_data_path = os.path.join(os.path.join(os.path.abspath(''), os.path.pardir), "assets")
speos_file = os.path.join(tests_data_path, os.path.join("LG_50M_Colorimetric_short.sv5", "LG_50M_Colorimetric_short.sv5"))

def clean_all_dbs(speos_client: core.SpeosClient):
    for item in (
        speos_client.jobs().list()
        + speos_client.scenes().list()
        + speos_client.simulation_templates().list()
        + speos_client.sensor_templates().list()
        + speos_client.source_templates().list()
        + speos_client.intensity_templates().list()
        + speos_client.spectrums().list()
        + speos_client.vop_templates().list()
        + speos_client.sop_templates().list()
        + speos_client.parts().list()
        + speos_client.bodies().list()
        + speos_client.faces().list()
    ):
        item.delete()

# Create connection with speos rpc server

In [ ]:
speos = core.Speos(host="localhost", port=50098)

# Scene
## Create empty scene

In [ ]:
my_scene = speos.client.scenes().create()

## Actions on a scene

### Load file

In [ ]:
my_scene.load_file(file_uri=speos_file)
print(my_scene)

### Get source ray path

Retrieve source ray paths for the first source - currently not working from a loaded file

In [ ]:
#my_scene_data = my_scene.get()
#assert len(my_scene_data.sources) > 0
#for ray_path in my_scene.get_source_ray_paths(source_path=my_scene.get().sources[0].name, rays_nb=2):
#    print(ray_path)

# Job
## Create a job for first simu

In [ ]:
job_message = core.Job(name="my_job")
job_message.scene_guid = my_scene.key
job_message.simulation_path = my_scene.get().simulations[0].name
job_message.direct_mc_simulation_properties.automatic_save_frequency = 1800
job_message.direct_mc_simulation_properties.stop_condition_rays_number = 200000

job_link = speos.client.jobs().create(message=job_message)

## Actions on a job

In [ ]:
job_link.start()

In [ ]:
job_link.get_state()

In [ ]:
while (
    job_link.get_state().state != job_message.State.FINISHED
    and job_link.get_state().state != job_message.State.STOPPED
    and job_link.get_state().state != job_message.State.IN_ERROR
    ):
        time.sleep(2)

job_link.get_progress_status()
job_link.get_results()

# Clean up

In [ ]:
job_link.delete()

In [ ]:
clean_all_dbs(speos.client)